In [0]:
sc



SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
transactions_df = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv", header = True, inferSchema = True)
transactions_df.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|
|7792c28f-2f9b-48b...|Adult Womens Sexy...|       adult|   shoes|  a

In [0]:
from pyspark.sql.functions import *
transactions_df.select([count(when(isnull(col), col)).alias(col) for col in transactions_df.columns]).show()

+-----------+-----------+-----+--------+-----------+-----------+----------------+------------------+--------+-----------------+-----------+----+-------------+----------+-------+
|ProductCode|Description|Brand|Category|SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|InvoiceDate|Mode|InvoiceNumber|CustomerId|Country|
+-----------+-----------+-----+--------+-----------+-----------+----------------+------------------+--------+-----------------+-----------+----+-------------+----------+-------+
|          0|          0|    0|       0|          0|          0|               0|                 0|       0|                0|          0|   0|            0|         0|      0|
+-----------+-----------+-----+--------+-----------+-----------+----------------+------------------+--------+-----------------+-----------+----+-------------+----------+-------+



In [0]:
amazon_df=transactions_df.withColumn("Year",year(transactions_df.InvoiceDate)).withColumn("Month",month(transactions_df.InvoiceDate))
amazon_df.show()


+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|Year|Month|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|2015|    1|
|7792c28f-2f9b-48b...|Ad

In [0]:
amazon_df.createOrReplaceTempView("Data_Table")

In [0]:
#a.	Find the top five performing mobile brands for the months of November, December and January r each year.
top_five=spark.sql("select Year,Brand,Month,count(*) as monthlySales from Data_Table where SubCategory='mobile' and Month in (1,11,12 ) group by Year,Brand,Month order by monthlysales desc").show()

+----+-------+-----+------------+
|Year|  Brand|Month|monthlySales|
+----+-------+-----+------------+
|2015|samsung|   12|         208|
|2015|samsung|   11|         183|
|2018|samsung|    1|         166|
|2017|samsung|    1|         153|
|2016|samsung|   11|         153|
|2015|samsung|    1|         151|
|2017|samsung|   11|         148|
|2016|samsung|    1|         144|
|2016|samsung|   12|         142|
|2017|samsung|   12|         123|
|2015|    blu|   12|          95|
|2015|  apple|    1|          89|
|2015|    blu|   11|          86|
|2016|  apple|    1|          82|
|2015|  nokia|   12|          80|
|2015|  apple|   11|          80|
|2016|    blu|   11|          80|
|2017|    blu|   11|          77|
|2015|  nokia|   11|          72|
|2018|    blu|    1|          72|
+----+-------+-----+------------+
only showing top 20 rows



In [0]:
#b.	Find the top five selling products for the months of November, December and January for each year.
top_five_sales=spark.sql("select Year,Brand,Category,Month,count(*) as monthSales from Data_Table where  Month in (1,11,12 ) group by Year,Brand,Category,Month order by monthsales desc").show()

+----+-------+-----------+-----+----------+
|Year|  Brand|   Category|Month|monthSales|
+----+-------+-----------+-----+----------+
|2017|   nike|   clothing|   11|       426|
|2016|   nike|   clothing|   11|       382|
|2016|   nike|   clothing|   12|       369|
|2015|   nike|   clothing|    1|       349|
|2015|   nike|   clothing|   11|       326|
|2016|   nike|   clothing|    1|       326|
|2018|   nike|   clothing|    1|       317|
|2015|   nike|   clothing|   12|       315|
|2017|   nike|   clothing|    1|       285|
|2017|   nike|   clothing|   12|       283|
|2015|samsung|electronics|   12|       225|
|2015|samsung|electronics|   11|       198|
|2018|samsung|electronics|    1|       176|
|2016|samsung|electronics|   11|       169|
|2017|samsung|electronics|    1|       167|
|2017|samsung|electronics|   11|       163|
|2015|samsung|electronics|    1|       159|
|2016|samsung|electronics|    1|       153|
|2016|samsung|electronics|   12|       153|
|2017|samsung|electronics|   12|

In [0]:
amazon_df1=amazon_df.withColumn("Quarter",quarter(amazon_df.InvoiceDate))
amazon_df1.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|Year|Month|Quarter|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+-------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|2015|    1| 

In [0]:

amazon_df1.createOrReplaceTempView("Data_Table1")

In [0]:
#2.	Give valuable insights to the investors
#a.	Number of products sold along with revenue aggregated quarterly
quarter=spark.sql("select Year,Quarter,sum(quantity) as totalbuys ,sum(TransactionAmount) as amount from Data_Table1 group by Year,Quarter order by Year,Quarter desc").show()

+----+-------+---------+-------------------+
|Year|Quarter|totalbuys|             amount|
+----+-------+---------+-------------------+
|2015|      4|    45636|       3.22182882E8|
|2015|      3|    43224|       3.17578576E8|
|2015|      2|    42069|2.23020302445285E14|
|2015|      1|    42693|       3.08548555E8|
|2016|      4|    42684|4.46040321418626E14|
|2016|      3|    44353|6.69060331721781E14|
|2016|      2|    41872|4.46040332280323E14|
|2016|      1|    49125|2.23020349928595E14|
|2017|      4|    45000|4.46040344731496E14|
|2017|      3|    40429|       2.91143349E8|
|2017|      2|    44283|       3.45020711E8|
|2017|      1|    44917|       3.23175489E8|
|2018|      2|     3905|        3.0780533E7|
|2018|      1|    42619|6.69060301392097E14|
+----+-------+---------+-------------------+



In [0]:
#b.	Number of products sold in each category aggregated quarterly
quarter_category=spark.sql("select Year,Quarter,Category,sum(quantity) as totalbuys  from Data_Table1 group by Year,Quarter,Category order by Year,Quarter desc").show()

+----+-------+-------------+---------+
|Year|Quarter|     Category|totalbuys|
+----+-------+-------------+---------+
|2015|      4|     clothing|    33537|
|2015|      4|  electronics|     9917|
|2015|      4|    furniture|        3|
|2015|      4|        shoes|     2179|
|2015|      3|        shoes|     2210|
|2015|      3|  electronics|     7497|
|2015|      3|     clothing|    33517|
|2015|      2|     clothing|    32481|
|2015|      2|        shoes|     2154|
|2015|      2|  electronics|     7434|
|2015|      1|     clothing|    32683|
|2015|      1|  electronics|     7872|
|2015|      1|        shoes|     2134|
|2015|      1|apple carplay|        4|
|2016|      4|     clothing|    32097|
|2016|      4|        shoes|     1834|
|2016|      4|    furniture|        6|
|2016|      4|  electronics|     8743|
|2016|      4|apple carplay|        4|
|2016|      3|    furniture|        6|
+----+-------+-------------+---------+
only showing top 20 rows



In [0]:
#c.	A number of products sold for each brand aggregated quarterly.
quarter_brand=spark.sql("select Year,Quarter,Brand,sum(quantity) as totalbuys  from Data_Table1 group by Year,Quarter,Brand order by Year,Quarter desc").show()

+----+-------+--------------------+---------+
|Year|Quarter|               Brand|totalbuys|
+----+-------+--------------------+---------+
|2015|      4|                nano|        5|
|2015|      4|               ariat|       23|
|2015|      4|              huawei|      206|
|2015|      4|             hisense|       16|
|2015|      4|             gameday|      144|
|2015|      4|                 lee|        9|
|2015|      4|              giamia|        3|
|2015|      4|        coveryouhair|        2|
|2015|      4|               magic|        9|
|2015|      4|              vionic|       49|
|2015|      4|             samsung|     1536|
|2015|      4|dunlop stonerite ...|       11|
|2015|      4|           lifeproof|       12|
|2015|      4|                just|       47|
|2015|      4|              dupont|       15|
|2015|      4|             dickies|      131|
|2015|      4|               nexus|       11|
|2015|      4|               texbo|        3|
|2015|      4|                lady

In [0]:
#3.	Find products that generated profit
#a.	Top 5 profit margin generating products for a given month.

spark.sql("select Brand, SubCategory,Marginpercentage,Month,Year from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Marginpercentage desc ) as rn" + " from Data_Table1) tmp where rn <= 5" ).show()

+---------+-----------+----------------+-----+----+
|    Brand|SubCategory|Marginpercentage|Month|Year|
+---------+-----------+----------------+-----+----+
|    bella|accessories|              30|    3|2015|
|  samsung|accessories|              30|    3|2015|
|    bella|accessories|              30|    3|2015|
|  samsung|accessories|              30|    3|2016|
|    black|accessories|              30|    3|2016|
| unlisted|accessories|              30|    4|2015|
|     drew|accessories|              20|    4|2015|
|blackhawk|accessories|              20|    4|2017|
|     drew|accessories|              20|    4|2017|
|  crystal|accessories|              10|    4|2015|
|    bella|accessories|              30|    6|2015|
|   unique|accessories|              30|    6|2016|
|   unique|accessories|              30|    6|2016|
| unlisted|accessories|              30|    6|2016|
|   unique|accessories|              30|    6|2017|
|  samsung|accessories|              30|    9|2016|
|  samsung|a

In [0]:
#4.	Brands that need marketing
#a.	Brand wise segregation of best performing categories profits wise
spark.sql("select Brand,Category,sum((Marginpercentage-Discountpercentage)*ActualPrice) as profit from Data_Table1 group by Brand,Category   order by profit desc").show()

+----------------+-----------+------------------+
|           Brand|   Category|            profit|
+----------------+-----------+------------------+
|         pioneer|electronics|8.6729999828087E15|
|           annie|   clothing|        8.761704E8|
|           yu&yu|      shoes|        1.585122E8|
|       panasonic|electronics|         4.87746E7|
|           ralph|   clothing|         4.30164E7|
|           cobra|electronics|         3.46752E7|
|       alienware|electronics|         3.43217E7|
|       unbranded|   clothing|         3.11234E7|
|             svs|electronics|          2.9155E7|
|        insignia|electronics|           2.604E7|
|            zigi|   clothing|         2.50264E7|
|interconvertible|electronics|           1.925E7|
|            muka|   clothing|         1.85255E7|
|          argyle|   clothing|          1.8095E7|
|           magic|   clothing|         1.70331E7|
|       australia|   clothing|         1.60755E7|
|          leagoo|electronics|           1.456E7|


In [0]:
#b.	Brand wise segregation of least performing categories orders wise
spark.sql("select Brand,Category,sum(Quantity) as Quantity_num from Data_Table1 group by Brand,Category   order by Quantity_num").show()

+----------------+-----------+------------+
|           Brand|   Category|Quantity_num|
+----------------+-----------+------------+
|             his|   clothing|           1|
|       bonnnibel|   clothing|           1|
|         luckers|   clothing|           1|
|          parisa|      shoes|           1|
|            male|   clothing|           1|
|jeffrey campbell|   clothing|           1|
|            doro|electronics|           1|
|          cooler|electronics|           1|
|           knife|   clothing|           1|
|          cliffs|   clothing|           1|
|           irish|   clothing|           1|
|       superfeet|      shoes|           1|
|          kendal|   clothing|           1|
|            loro|   clothing|           1|
|          castle|   clothing|           1|
|         toshiba|   clothing|           1|
|         elliott|      shoes|           1|
|      highstreet|   clothing|           1|
|       muckboots|   clothing|           1|
|            idea|   clothing|  

In [0]:
#5.	Find the products that should be part of the lighting deals.
#a.	Find the top five selling products in a given month

spark.sql("select Brand,Category,SubCategory,Month,Quantity from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Quantity desc ) as rn" + " from Data_Table1) tmp where rn <= 5" ).show()

+---------+-----------+-----------+-----+--------+
|    Brand|   Category|SubCategory|Month|Quantity|
+---------+-----------+-----------+-----+--------+
|     nine|   clothing|accessories|    3|       4|
|     nine|   clothing|accessories|    3|       4|
|     nine|   clothing|accessories|    3|       4|
|   lenovo|electronics|accessories|    3|       4|
|     nine|   clothing|accessories|    3|       4|
| unlisted|   clothing|accessories|    4|       4|
|     nine|   clothing|accessories|    4|       4|
|     nine|   clothing|accessories|    4|       4|
|  crystal|   clothing|accessories|    4|       3|
|  allegra|   clothing|accessories|    4|       2|
|     nine|   clothing|accessories|    6|       4|
| unlisted|   clothing|accessories|    6|       4|
|     nine|   clothing|accessories|    6|       4|
|   lenovo|electronics|accessories|    6|       4|
|     nine|   clothing|accessories|    6|       4|
|     nine|   clothing|accessories|    9|       4|
|  crystal|   clothing|accessor

In [0]:
#b.	Find the top five products having the highest margin in a given month

spark.sql("select Brand,Category,SubCategory,Month,Marginpercentage from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Marginpercentage desc ) as rn" + " from Data_Table1) tmp where rn <= 5" ).show()

+---------+-----------+-----------+-----+----------------+
|    Brand|   Category|SubCategory|Month|Marginpercentage|
+---------+-----------+-----------+-----+----------------+
|    bella|   clothing|accessories|    3|              30|
|  samsung|electronics|accessories|    3|              30|
|    bella|   clothing|accessories|    3|              30|
|  samsung|electronics|accessories|    3|              30|
|    black|   clothing|accessories|    3|              30|
| unlisted|   clothing|accessories|    4|              30|
|     drew|   clothing|accessories|    4|              20|
|blackhawk|   clothing|accessories|    4|              20|
|     drew|   clothing|accessories|    4|              20|
|  crystal|   clothing|accessories|    4|              10|
|    bella|   clothing|accessories|    6|              30|
|   unique|   clothing|accessories|    6|              30|
|   unique|   clothing|accessories|    6|              30|
| unlisted|   clothing|accessories|    6|              3

In [0]:
#c.	Find the five lowest selling products in a given month.

spark.sql("select Brand,Category,SubCategory,Month,Quantity from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Quantity ) as rn" + " from Data_Table1) tmp where rn <= 5" ).show()

+-----------------+-----------+-----------+-----+--------+
|            Brand|   Category|SubCategory|Month|Quantity|
+-----------------+-----------+-----------+-----+--------+
|chubbychicocharms|   clothing|accessories|    3|       1|
|chubbychicocharms|   clothing|accessories|    3|       1|
|            bella|   clothing|accessories|    3|       2|
|            qupid|   clothing|accessories|    3|       2|
|          samsung|electronics|accessories|    3|       2|
|          allegra|   clothing|accessories|    4|       2|
|             drew|   clothing|accessories|    4|       2|
|          allegra|   clothing|accessories|    4|       2|
|        panasonic|electronics|accessories|    4|       2|
|        blackhawk|   clothing|accessories|    4|       2|
|        panasonic|electronics|accessories|    6|       2|
|            bella|   clothing|accessories|    6|       2|
|          allegra|   clothing|accessories|    6|       2|
|            qupid|   clothing|accessories|    6|       

In [0]:
#6.	Trending brands for given month order wise
spark.sql("select Brand,SubCategory,Month,sum(Quantity) as Quantity_num from Data_Table1 group by Brand,Month,Subcategory order by Month,Quantity_num desc").show()

+----------+-----------------+-----+------------+
|     Brand|      SubCategory|Month|Quantity_num|
+----------+-----------------+-----+------------+
|   samsung|           mobile|    1|        1567|
|      nike|      men's shoes|    1|         941|
|     apple|           mobile|    1|         772|
|       blu|           mobile|    1|         680|
|     nokia|           mobile|    1|         589|
|        lg|           mobile|    1|         497|
|       htc|           mobile|    1|         472|
|  motorola|           mobile|    1|         390|
|      sony|           mobile|    1|         385|
|blackberry|           mobile|    1|         336|
|      nike|all women's shoes|    1|         319|
|      puma|      men's shoes|    1|         274|
|      nike|  shoes & jewelry|    1|         254|
|      nike|  all men's shoes|    1|         249|
|      vans|      men's shoes|    1|         223|
|     yu&yu|all women's shoes|    1|         217|
|       new|      men's shoes|    1|         211|
